In [71]:
import pandas as pd
import numpy as np

In [72]:
votes = pd.read_csv("https://raw.githubusercontent.com/jamilditter/election_results/refs/heads/main/data/countypres_2000-2024.csv")
unemployment = pd.read_excel("https://raw.github.com/jamilditter/election_results/refs/heads/main/data/LAUS_Annual_Unemployment_By_State_2000to2024.xlsx", skiprows=3)
med_income = pd.read_csv("https://raw.githubusercontent.com/jamilditter/election_results/refs/heads/main/data/Median_Household_Income_By_State_FRED.csv")
age = pd.read_csv("https://raw.githubusercontent.com/jamilditter/election_results/refs/heads/main/data/State_by_Age.csv")
education = pd.read_csv("https://raw.githubusercontent.com/jamilditter/election_results/refs/heads/main/data/State_Education_by_Degree.csv")
population = pd.read_csv("https://raw.githubusercontent.com/jamilditter/election_results/refs/heads/main/data/Population%20by%20Age%20and%20Sex%20-%20US%2C%20States%2C%20Counties.csv")
bach_2024 = pd.read_csv("https://raw.githubusercontent.com/jamilditter/election_results/refs/heads/main/data/FRED_BachelorsEduAttainment2024.csv")

In [73]:
med_income.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 52 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   observation_date   25 non-null     object
 1   MEHOINUSALA672N    25 non-null     int64 
 2   MEHOINUSARA672N    25 non-null     int64 
 3   MEHOINUSAZA672N    25 non-null     int64 
 4   MEHOINUSCAA672N    25 non-null     int64 
 5   MEHOINUSCOA672N    25 non-null     int64 
 6   MEHOINUSCTA672N    25 non-null     int64 
 7   MEHOINUSDCA672N    25 non-null     int64 
 8   MEHOINUSDEA672N    25 non-null     int64 
 9   MEHOINUSFLA672N    25 non-null     int64 
 10  MEHOINUSGAA672N    25 non-null     int64 
 11  MEHOINUSHIA672N    25 non-null     int64 
 12  MEHOINUSIAA672N    25 non-null     int64 
 13  MEHOINUSIDA672N    25 non-null     int64 
 14  MEHOINUSILA672N    25 non-null     int64 
 15  MEHOINUSINA672N    25 non-null     int64 
 16  MEHOINUSIAA672N.1  25 non-null     int64 
 17 

In [74]:
votes["totalvotes"] = votes.groupby(["state", "year"])["candidatevotes"].transform("sum")
votes["dem"] = votes[votes["party"] == "DEMOCRAT"].groupby(["state", "year"])["candidatevotes"].transform("sum")
votes["rep"] = votes[votes["party"] == "REPUBLICAN"].groupby(["state", "year"])["candidatevotes"].transform("sum")
votes["other"] = votes[votes["party"] == "OTHER"].groupby(["state", "year"])["candidatevotes"].transform("sum")
votes["green"] = votes[votes["party"] == "GREEN"].groupby(["state", "year"])["candidatevotes"].transform("sum")
votes.head()

totalvotes_melted = pd.melt(
    votes, id_vars = ["state", "year"],
    value_vars = ["totalvotes"],
    var_name = "party",
    value_name = "total_votes"
).drop_duplicates().dropna().drop(columns=["party"])

demvotes_melted = pd.melt(
    votes, id_vars = ["state", "year"],
    value_vars = ["dem"],
    var_name = "party",
    value_name = "dem_votes"
).drop_duplicates().dropna().drop(columns=["party"])

repvotes_melted = pd.melt(
    votes, id_vars = ["state", "year"],
    value_vars = ["rep"],
    var_name = "party",
    value_name = "rep_votes"
).drop_duplicates().dropna().drop(columns=["party"])

othervotes_melted = pd.melt(
    votes, id_vars = ["state", "year"],
    value_vars = ["other"],
    var_name = "party",
    value_name = "other_votes"
).drop_duplicates().dropna().drop(columns=["party"])

greenvotes_melted = pd.melt(
    votes, id_vars = ["state", "year"],
    value_vars = ["green"],
    var_name = "party",
    value_name = "green_votes"
).drop_duplicates().dropna().drop(columns=["party"])

repvotes_melted.head()

,state,year,rep_votes
1,ALABAMA,2000,944409.0
269,ALASKA,2000,167398.0
433,ARIZONA,2000,781652.0
493,ARKANSAS,2000,472940.0
793,CALIFORNIA,2000,4567429.0


In [75]:
rename_states = {
    'LAUST010000000000003': 'Alabama',
    'LAUST020000000000003': 'Alaska',
    'LAUST040000000000003': 'Arizona',
    'LAUST050000000000003': 'Arkansas',
    'LAUST060000000000003': 'California',
    'LAUST080000000000003': 'Colorado',
    'LAUST090000000000003': 'Connecticut',
    'LAUST100000000000003': 'Delaware',
    'LAUST110000000000003': 'District of Columbia',
    'LAUST120000000000003': 'Florida',
    'LAUST130000000000003': 'Georgia',
    'LAUST150000000000003': 'Hawaii',
    'LAUST160000000000003': 'Idaho',
    'LAUST170000000000003': 'Illinois',
    'LAUST180000000000003': 'Indiana',
    'LAUST190000000000003': 'Iowa',
    'LAUST200000000000003': 'Kansas',
    'LAUST210000000000003': 'Kentucky',
    'LAUST220000000000003': 'Louisiana',
    'LAUST230000000000003': 'Maine',
    'LAUST240000000000003': 'Maryland',
    'LAUST250000000000003': 'Massachusetts',
    'LAUST260000000000003': 'Michigan',
    'LAUST270000000000003': 'Minnesota',
    'LAUST280000000000003': 'Mississippi',
    'LAUST290000000000003': 'Missouri',
    'LAUST300000000000003': 'Montana',
    'LAUST310000000000003': 'Nebraska',
    'LAUST320000000000003': 'Nevada',
    'LAUST330000000000003': 'New Hampshire',
    'LAUST340000000000003': 'New Jersey',
    'LAUST350000000000003': 'New Mexico',
    'LAUST360000000000003': 'New York',
    'LAUST370000000000003': 'North Carolina',
    'LAUST380000000000003': 'North Dakota',
    'LAUST390000000000003': 'Ohio',
    'LAUST400000000000003': 'Oklahoma',
    'LAUST410000000000003': 'Oregon',
    'LAUST420000000000003': 'Pennsylvania',
    'LAUST440000000000003': 'Rhode Island',
    'LAUST450000000000003': 'South Carolina',
    'LAUST460000000000003': 'South Dakota',
    'LAUST470000000000003': 'Tennessee',
    'LAUST480000000000003': 'Texas',
    'LAUST490000000000003': 'Utah',
    'LAUST500000000000003': 'Vermont',
    'LAUST510000000000003': 'Virginia',
    'LAUST530000000000003': 'Washington',
    'LAUST540000000000003': 'West Virginia',
    'LAUST550000000000003': 'Wisconsin',
    'LAUST560000000000003': 'Wyoming'
}

unemployment["Series ID"] = unemployment["Series ID"].replace(rename_states)

unemployment_renamed = unemployment.rename(columns = lambda x: x[-4:]).rename(columns={"s ID": "state"})

unemployment_melted = pd.melt(
    unemployment_renamed, id_vars = ["state"],
    value_vars = ["2000", "2004", "2008", "2012", "2016", "2020", "2024"],
    var_name = "year",
    value_name = "unemployment_rate"
)

election_years = ["2000", "2004", "2008", "2012", "2016", "2020", "2024"]
unemployment_melted = unemployment_melted[unemployment_melted["year"].isin(election_years)]
unemployment_melted["year"].unique()

array(['2000', '2004', '2008', '2012', '2016', '2020', '2024'],
      dtype=object)

In [83]:
abbr_to_state = {
    "ALA": "Alabama",
    "ARA": "Arkansas",
    "AZA": "Arizona",
    "CAA": "California",
    "COA": "Colorado",
    "CTA": "Connecticut",
    "DCA": "District of Columbia",
    "DEA": "Delaware",
    "FLA": "Florida",
    "GAA": "Georgia",
    "HIA": "Hawaii",
    "IDA": "Idaho",
    "ILA": "Illinois",
    "INA": "Indiana",
    "IOA": "Iowa",
    "KSA": "Kansas",
    "KYA": "Kentucky",
    "LAA": "Louisiana", 
    "MEA": "Maine",
    "MDA": "Maryland",
    "MAA": "Massachusetts",
    "MIA": "Michigan",
    "MNA": "Minnesota",
    "MSA": "Mississippi",
    "MOA": "Missouri",
    "MTA": "Montana",
    "NBA": "Nebraska",
    "NVA": "Nevada",
    "NHA": "New Hampshire",
    "NJA": "New Jersey",
    "NMA": "New Mexico",
    "NYA": "New York",
    "NCA": "North Carolina",
    "NDA": "North Dakota",
    "OHA": "Ohio",
    "OKA": "Oklahoma",
    "ORA": "Oregon",
    "PAA": "Pennsylvania",
    "RIA": "Rhode Island",
    "SCA": "South Carolina",
    "SDA": "South Dakota",
    "TNA": "Tennessee",
    "TXA": "Texas",
    "UTA": "Utah",
    "VAA": "Virginia",
    "VTA": "Vermont",
    "WAA": "Washington",
    "WIA": "Wisconsin",
    "WVA": "West Virginia",
    "WYA": "Wyoming"
}

med_income["observation_date"] = pd.to_datetime(med_income["observation_date"]).dt.year
med_income_renamed = med_income.rename(columns={"observation_date": "year"})

med_income_melted = pd.melt(
    med_income_renamed, id_vars = ["year"],
    value_vars = ['MEHOINUSALA672N', 'MEHOINUSARA672N', 'MEHOINUSAZA672N', 'MEHOINUSCAA672N', 'MEHOINUSCOA672N', 
                  'MEHOINUSCTA672N', 'MEHOINUSDCA672N', 'MEHOINUSDEA672N', 'MEHOINUSFLA672N', 'MEHOINUSGAA672N', 
                  'MEHOINUSHIA672N', 'MEHOINUSIAA672N', 'MEHOINUSIDA672N', 'MEHOINUSILA672N', 'MEHOINUSINA672N', 'MEHOINUSIAA672N.1', 
                  'MEHOINUSKSA672N', 'MEHOINUSKYA672N', 'MEHOINUSLAA672N', 'MEHOINUSMAA672N', 'MEHOINUSMDA672N', 
                  'MEHOINUSMEA672N', 'MEHOINUSMIA672N', 'MEHOINUSMNA672N', 'MEHOINUSMOA672N', 'MEHOINUSMSA672N', 
                  'MEHOINUSMTA672N', 'MEHOINUSNEA672N','MEHOINUSNCA672N', 'MEHOINUSNDA672N', 'MEHOINUSNEA672N.1', 'MEHOINUSNHA672N', 
                  'MEHOINUSNJA672N', 'MEHOINUSNMA672N', 'MEHOINUSNVA672N', 'MEHOINUSNYA672N', 'MEHOINUSOHA672N', 
                  'MEHOINUSOKA672N', 'MEHOINUSORA672N', 'MEHOINUSPAA672N', 'MEHOINUSSCA672N', 'MEHOINUSSDA672N', 
                  'MEHOINUSTNA672N', 'MEHOINUSTXA672N', 'MEHOINUSUTA672N', 'MEHOINUSVAA672N', 'MEHOINUSVTA672N', 
                  'MEHOINUSWAA672N', 'MEHOINUSWIA672N', 'MEHOINUSWVA672N', 'MEHOINUSWYA672N'],
    var_name = "state",
    value_name = "median_income"
)

med_income_melted["state"] = med_income_melted["state"].str[8:11].map(abbr_to_state)
med_income_melted["year"] = med_income_melted["year"].astype(str)

med_income_melted = med_income_melted[med_income_melted["year"].isin(election_years)]
med_income_melted["year"].unique()

med_income_melted.head()

,year,state,median_income


In [58]:
fips_to_states = {
    1: "Alabama",
    2: "Alaska",
    4: "Arizona",
    5: "Arkansas",
    6: "California",
    8: "Colorado",
    9: "Connecticut",
    10: "Delaware",
    11: "District of Columbia",
    12: "Florida",
    13: "Georgia",
    15: "Hawaii",
    16: "Idaho",
    17: "Illinois",
    18: "Indiana",
    19: "Iowa",
    20: "Kansas",
    21: "Kentucky",
    22: "Louisiana",
    23: "Maine",
    24: "Maryland",
    25: "Massachusetts",
    26: "Michigan",
    27: "Minnesota",
    28: "Mississippi",
    29: "Missouri",
    30: "Montana",
    31: "Nebraska",
    32: "Nevada",
    33: "New Hampshire",
    34: "New Jersey",
    35: "New Mexico",
    36: "New York",
    37: "North Carolina",
    38: "North Dakota",
    39: "Ohio",
    40: "Oklahoma",
    41: "Oregon",
    42: "Pennsylvania",
    44: "Rhode Island",
    45: "South Carolina",
    46: "South Dakota",
    47: "Tennessee",
    48: "Texas",
    49: "Utah",
    50: "Vermont",
    51: "Virginia",
    53: "Washington",
    54: "West Virginia",
    55: "Wisconsin",
    56: "Wyoming"
}

age["STATEFIP"] = age["STATEFIP"].replace(fips_to_states)
age = age.rename(columns = {"STATEFIP": "state", "YEAR": "year"})

med_age = age.loc[:, ["state", "year", "med_age"]]

In [59]:
education.head()

,STATEFIP,YEAR,total_pop,bach_pop,hs_pop,assoc_pop,masters_pop,drs_pop
0,1,2000,1531294,366846,834824,162140,145153,22331
1,1,2004,1668813,398768,884667,198097,161298,25983
2,1,2008,1799699,420962,979366,204430,169564,25377
3,1,2012,893318,450959,0,227969,185956,28434
4,1,2016,989675,487746,0,260917,208372,32640


In [60]:
education["STATEFIP"] = education["STATEFIP"].replace(fips_to_states)
education = education.rename(columns = {"STATEFIP": "state", "YEAR": "year"})

education["bach_per"] = education["bach_pop"] / education["total_pop"]
education["hs_per"] = education["hs_pop"] / education["total_pop"]

bach_2024["year"] = 2024
bach_2024 = bach_2024.rename(columns = {"State": "state"})

education_merge = pd.merge(education, bach_2024, on=["state", "year"], how="outer")
education_merge.loc[education_merge["year"] == 2024, "bach_per"] = education_merge.loc[education_merge["year"] == 2024, "2024 Bachelors or Higher"]
education_merge = education_merge.drop(columns = ["2024 Bachelors or Higher"])

edu_bach = education_merge.loc[:, ["state", "year", "bach_per"]]
edu_hs = education_merge.loc[:, ["state", "year", "hs_per"]]

edu_bach.head()

,state,year,bach_per
0,Alabama,2000,0.239566
1,Alabama,2004,0.238953
2,Alabama,2008,0.233907
3,Alabama,2012,0.504814
4,Alabama,2016,0.492835


In [61]:
population_renamed = population.rename(columns = {"Total Population": "total_pop", "Year": "year", "Description": "state"})
population_renamed["voting_age_pop"] = population_renamed.groupby(["state", "year"])[["Population 18-54", "Population 55+"]].transform("sum").sum(axis=1)
population_renamed = population_renamed.loc[:, ["state", "year", "voting_age_pop", "Population 18-24", "Population 25-44", "Population 45-64", "Population 65+"]]
population_renamed = population_renamed.loc[population_renamed["state"] != "U.S."]
population_renamed["year"] = population_renamed["year"].astype(str)
population_renamed = population_renamed[population_renamed["year"].isin(election_years)]

population_renamed.head()

,state,year,voting_age_pop,Population 18-24,Population 25-44,Population 45-64,Population 65+
25,Alabama,2000,3329900.0,442244,1284371,1022405,580880
29,Alabama,2004,3417067.0,457934,1231571,1133736,593826
33,Alabama,2008,3583279.0,470431,1231410,1244494,636944
37,Alabama,2012,3699663.0,485538,1225621,1290419,698085
41,Alabama,2016,3766363.0,462098,1231213,1289047,784005


In [64]:
unemployment_melted["year"] = unemployment_melted["year"].astype(int)
unemployment_melted["state"] = unemployment_melted["state"].str.upper()
med_income_melted["year"] = med_income_melted["year"].astype(int)
med_income_melted["state"] = med_income_melted["state"].str.upper()
med_age["state"] = med_age["state"].str.upper()
edu_bach["state"] = edu_bach["state"].str.upper()
edu_hs["state"] = edu_hs["state"].str.upper()
population_renamed["state"] = population_renamed["state"].str.upper()
population_renamed["year"] = population_renamed["year"].astype(int)

merge1 = pd.merge(totalvotes_melted, demvotes_melted, on=["state", "year"], how="left")
merge2 = pd.merge(merge1, repvotes_melted, on=["state", "year"], how="left")
merge3 = pd.merge(merge2, othervotes_melted, on=["state", "year"], how="left")
merge4 = pd.merge(merge3, greenvotes_melted, on=["state", "year"], how="left")
merge5 = pd.merge(merge4, unemployment_melted, on=["state", "year"], how="left")
merge6 = pd.merge(merge5, med_income_melted, on=["state", "year"], how="left")
merge7 = pd.merge(merge6, med_age, on=["state", "year"], how="left")
merge8 = pd.merge(merge7, edu_bach, on=["state", "year"], how="left")
merge9 = pd.merge(merge8, edu_hs, on=["state", "year"], how="left")
merged_data = pd.merge(merge9, population_renamed, on=["state", "year"], how="left")

merged_data["dem_votes_per"] = merged_data["dem_votes"] / merged_data["total_votes"]
merged_data["rep_votes_per"] = merged_data["rep_votes"] / merged_data["total_votes"]
merged_data["other_votes_per"] = merged_data["other_votes"] / merged_data["total_votes"]
merged_data["green_votes_per"] = merged_data["green_votes"] / merged_data["total_votes"]
merged_data["voter_participation"] = merged_data["total_votes"] / merged_data["voting_age_pop"]

merged_data.head()

merged_data.to_csv("Voting Breakdown By Socioeconomic and Demographic Factors.csv")

In [65]:
merged_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 357 entries, 0 to 356
Data columns (total 22 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   state                357 non-null    object 
 1   year                 357 non-null    int64  
 2   total_votes          357 non-null    int64  
 3   dem_votes            357 non-null    float64
 4   rep_votes            357 non-null    float64
 5   other_votes          348 non-null    float64
 6   green_votes          83 non-null     float64
 7   unemployment_rate    357 non-null    float64
 8   median_income        329 non-null    float64
 9   med_age              306 non-null    float64
 10  bach_per             357 non-null    float64
 11  hs_per               306 non-null    float64
 12  voting_age_pop       357 non-null    float64
 13  Population 18-24     357 non-null    int64  
 14  Population 25-44     357 non-null    int64  
 15  Population 45-64     357 non-null    int

In [66]:
merged_data.isna().sum().to_frame(name='Number of Missing Values')

,Number of Missing Values
state,0
year,0
total_votes,0
dem_votes,0
rep_votes,0
other_votes,9
green_votes,274
unemployment_rate,0
median_income,28
med_age,51
